In [ ]:
# from google.colab import userdata

# ngrok_key = userdata.get('ngrok_key')
# line_access_token = userdata.get('line_access_token')
# line_secret = userdata.get('line_secret')
# port = 5000

In [1]:
import os

line_channel_access_token = os.environ.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = os.environ.get('LINE_CHANNEL_SECRET')
open_api_key = os.environ.get('OPENAI_API_KEY')
port = 5000

In [2]:
from openai import OpenAI

client = OpenAI(api_key=open_api_key)
def get_completion(prompt):
    messages = [{"role": "system", "content": "以下用繁體中文台灣用語回答"}, 
                {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-4o-2024-11-20",
        messages=messages,
        temperature=0,
        max_tokens=256
        
    )
    return response.choices[0].message.content

In [3]:
result = get_completion("簡介東吳大學")
print(result)

東吳大學（Soochow University）是台灣一所歷史悠久且具有深厚人文底蘊的私立大學，創立於1900年，最早設立於中國蘇州，為中國第一所西式大學。1951年，東吳大學在台灣復校，校本部位於台北市士林區，另有城中校區位於台北市中正區。

東吳大學以「養天地正氣，法古今完人」為校訓，重視全人教育，致力於培養學生的專業能力與人文素養。該校以法學院、商學院、文學院、理學院、外語學院、社會科學院及音樂學院等七大學院聞名，特別是法學院與商學院在台灣享有極高的聲譽。

東吳大學以小而精的教學模式著稱，師生比例適中，提供學生良好的學習環境與資源。此外，學校也積極推動


In [4]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = get_completion(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )            
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

        
@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Dec/2024 15:52:04] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}
BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"538379903663080146","quoteToken":"DRkNTYQn_olxgR9DLwzhQ-_LELs4F9wyksyTPkLg7kIjWW1iklf5fw3qWL6I8Zh6qEab39qsYEmSuNJAg--KVeWgJvIfngg-6JJQ1TWVd3f1vmnUlmSilypqqu6i3aErZ7jzPSjQl_G0NAiZ8xKEIg","text":"你好"},"webhookEventId":"01JEN6F3ERQBW7JG3PCVZ112E7","deliveryContext":{"isRedelivery":false},"timestamp":1733730733141,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"73fd36a5a1b4464a8099da35498d8259","mode":"active"}]}


127.0.0.1 - - [09/Dec/2024 15:52:13] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"538379916631867620","quoteToken":"qfLIWp33h_bBF3caCx0kXNVJS0atTwriUZ0AbR6yyWvJAzopss9Y1NtvT9Ja_0gxnREdzmK82tfJLQZh8hoT8tqcgeIAual6Ys602GKcGy6P19dD7Q9yWPT9Wd7kw0-jktOMmRPu50sCCEWAtlCRdw","text":"AI 你是誰？"},"webhookEventId":"01JEN6FB4CHNPVGNC9VW6E9FA2","deliveryContext":{"isRedelivery":false},"timestamp":1733730740952,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"4d0ccd23e8ae4b9b908e4d55199b9510","mode":"active"}]}


...

127.0.0.1 - - [09/Dec/2024 15:52:49] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"538380008923333123","quoteToken":"wFt7dphDt-IeshvXx5miEbBUp1RObvj6w3LBra4qy2zo1CTVbGCVpGd0pn30FLR9wKTLCHbGAUPQxWFmZFNMgqRt_7nMQJF8Brsmmk8KFlDV7K5h9Oni32qXMYFqRh3MhCrjiCXbbDg9XwlvjLIeig","text":"城區部地址？"},"webhookEventId":"01JEN6H0N9WTP3B8DX715RJMZF","deliveryContext":{"isRedelivery":false},"timestamp":1733730795915,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"240ae1614f284bd5a1e4c5dab52930b8","mode":"active"}]}


127.0.0.1 - - [09/Dec/2024 15:53:16] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"538380020986151428","quoteToken":"cXKPlVnSHY-6RZB9MRVsv_lKZiF4WTCFTYNI-Sk1vkfASvYytFCKX9oWF4ak6CSPvfKj0j0IgfRTmUvmPBKr9sVVAUNR4vcJe54KHcn0IM_90-Yh0abZuxnB1joZWjIZ8lcPiljduLLx1GVYs6AJkw","text":"AI 城區部地址"},"webhookEventId":"01JEN6H7RQB0GC6GQPMGBXQG3C","deliveryContext":{"isRedelivery":false},"timestamp":1733730803094,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"0ffd678064524018a2e0fc28be40b72b","mode":"active"}]}


127.0.0.1 - - [09/Dec/2024 15:53:24] "POST / HTTP/1.1" 200 -
